# Introduction

In this project, we will leverage the power of BigQuery, Google Cloud's fully-managed data warehouse, to extract valuable insights from vast amounts of data.

The ability to understand customer behavior, track conversions, and optimize the performance of an online store is crucial for businesses to thrive in the digital era. With Google Analytics providing a wealth of information about website visitors, traffic sources, and user interactions, we can uncover hidden patterns and extract actionable insights for stakeholders.

In this notes, I'll evaluate the performance of an online store using Google Analytics data. We aim to answer critical questions such as:

- What are the main traffic sources driving visitors to the online store?
- How do user demographics and behaviors impact conversion rates?
- What are the most visited pages in the website?
- Are there any seasonal or daily trends or patterns in website traffic and sales?
- Can we identify any potential areas for improvement in the user experience or marketing strategies?

To answer these questions, we will use BigQuery, which allows us to handle large-scale datasets efficiently and perform complex queries seamlessly. By combining SQL queries, data visualization, and exploratory analysis techniques, we will gain valuable insights into the online store's performance and make data-driven recommendations for optimization.

In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [200]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [201]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [202]:
key_path = './service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [203]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

In [204]:
# Try if this connected

query = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170701`
LIMIT 5
"""

df = client.query(query).to_dataframe()
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           5 non-null      Int64 
 2   visitId               5 non-null      Int64 
 3   visitStartTime        5 non-null      Int64 
 4   date                  5 non-null      object
 5   totals                5 non-null      object
 6   trafficSource         5 non-null      object
 7   device                5 non-null      object
 8   geoNetwork            5 non-null      object
 9   customDimensions      5 non-null      object
 10  hits                  5 non-null      object
 11  fullVisitorId         5 non-null      object
 12  userId                0 non-null      object
 13  clientId              0 non-null      object
 14  channelGrouping       5 non-null      object
 15  socialEngagementType  5 non-null      object

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,2,1498921578,1498921578,20170701,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",9117776942963408674,None,None,Direct,Not Socially Engaged
1,<NA>,1,1498934175,1498934175,20170701,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Opera Mini', 'browserVersion': 'n...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",734032105411402836,None,None,Organic Search,Not Socially Engaged
2,<NA>,1,1498950889,1498950889,20170701,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Android Webview', 'browserVersion...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 16, 'minu...",5696457532157595544,None,None,Social,Not Socially Engaged
3,<NA>,1,1498939911,1498939911,20170701,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",5009259967134102270,None,None,Social,Not Socially Engaged
4,<NA>,1,1498945129,1498945129,20170701,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Safari (in-app)', 'browserVersion...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 14, 'minu...",2881134828551603619,None,None,Social,Not Socially Engaged


Explanation about fields: 
https://support.google.com/analytics/answer/3437719?hl=en

**Main Traffic Sources**

By checking the main traffic sources, we can find out more about the store's approach to marketing and how well or how bad some sources are performing.

In [205]:
# Main traffic sources last month (July 2017)

query = """
    SELECT channelGrouping, COUNT(visitId) number_of_visits
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
    GROUP BY channelGrouping
    HAVING number_of_visits > 0
    ORDER BY number_of_visits DESC
"""

df = client.query(query).to_dataframe()

fig = px.bar(df, x='channelGrouping', y='number_of_visits',
            title='Number of visit from various channel',
            labels={'number_of_visits': 'Number of Visits',
                    'channelGrouping': 'Channel Group'})

fig.show()

In [206]:
# Most visited pages

query = """
SELECT
  SUBSTRING(hit.page.pagePath, INSTR(hit.page.pagePath, '/', -1) + 1) page,
  COUNT(hit.page.pagePath) number_of_visits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h, UNNEST(h.hits) hit
WHERE _TABLE_SUFFIX BETWEEN '20170101' AND '20170731'
GROUP BY page
ORDER BY number_of_visits DESC
"""

df = client.query(query).to_dataframe()

fig = px.bar(df.iloc[1:21, :], x='page', y='number_of_visits',
            title='Number of visit from various pages',
            labels={'number_of_visits': 'Number of Visits', 
                    'page': 'Page'})

fig.show()

NOTE:
- Use UNNEST to Access Nested Fields
> When dealing with arrays of structs, you need to use the UNNEST function to flatten the array. 

**Demographics and Behaviors Impact Conversion Rates**

In [207]:
# Demographics and behaviors impact conversion rates

query = """
    SELECT 
      geoNetwork.continent region,
      SUM(totals.transactions) total_transactions,
      COUNT(totals.pageviews) total_views,
      COUNT(totals.bounces) total_bounces,
      ROUND(AVG(totals.transactions), 2) avg_transactions,
      ROUND(AVG(totals.pageviews), 2) avg_views,
      ROUND(SUM(totals.transactions) / COUNT(totals.pageviews), 2) * 100 views_to_transactions
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
    GROUP BY geoNetwork.continent
    ORDER BY total_views DESC
"""

df = client.query(query).to_dataframe()
df = df.fillna(0)

# Create subplots: 1 row, 2 columns
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Total Transactions by Region", "Total Views by Region"),
                    specs=[[{'type': 'pie'}, {'type': 'pie'}]])

# Pie chart for total views by region
fig.add_trace(go.Pie(labels=df['region'], values=df['total_views']), row=1, col=1)

# Pie chart for total bounces by region
fig.add_trace(go.Pie(labels=df['region'], values=df['total_transactions']), row=1, col=2)

# Show the plot
fig.show()

In [208]:
# Create subplots: 1 row, 2 columns
fig = make_subplots(rows=1, cols=3, subplot_titles=("Average Transactions by Region", "Average Views by Region", "Views to Transaction by Region"))

# Add the total transactions bar plot
fig.add_trace(
    go.Bar(x=df['region'], y=df['avg_transactions'], name='Average Transactions', marker_color='blue'),
    row=1, col=1
)

# Add the total views bar plot
fig.add_trace(
    go.Bar(x=df['region'], y=df['avg_views'], name='Average Views', marker_color='green'),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=df['region'], y=df['views_to_transactions'], name='Views to Transactions', marker_color='orange'),
    row=1, col=3
)

# Update layout
fig.update_layout(
    title_text='Average Transactions/Views and Views to Transaction per Region',
    showlegend=False
)

# Update x-axis and y-axis titles
fig.update_xaxes(title_text='Region', row=1, col=1)
fig.update_yaxes(title_text='Average Transactions', row=1, col=1)
fig.update_xaxes(title_text='Region', row=1, col=2)
fig.update_yaxes(title_text='Average Views', row=1, col=2)
fig.update_xaxes(title_text='Region', row=1, col=3)
fig.update_yaxes(title_text='Views to Transactions', row=1, col=3)

# Show the plot
fig.show()

As we can see in our previous charts depicting views and transactions per region:

- **International Audience:** Although North America is the dominant share of views and transactions, views from Europe and Asia are also significant, suggesting there is a demand or attention from these regions to the store.

- **Imbalance between Views and Transactions:** This is likely to happen if a certain product or service is focused primarily on the US market, but depending on the product or service, expanding the audience might be a simple question of adding new payment methods or outsourcing delviery.

In sum, depending on the objective or strategic goals of the store, it might be worth considering whether or not to expand business to attend other regions.

**Daily Trends**

Doing this can help us understand better the user behavior and might provide some insights about the best time for creating social media posts with content that might attract more users, such as promotions and discounts.

NOTES:

- Bounce : A situation where site visitors just come and don't take any action.

> In Analytics, a bounce is calculated specifically as a session that triggers only a single request to the Analytics server, such as when a user opens a single page on your site and then exits without triggering any other requests to the Analytics server during that session.

- Bounce rate: Percentage of all sessions on your site in which users viewed only a single page and triggered only a single request to the Analytics server.



In [209]:
# Daily Trends

query = """
SELECT
  hit.hour hour,
  COUNT(*) total_visits,
  SUM(totals.bounces) total_bounces,
  (SUM(totals.bounces) / COUNT(*) * 100) bounce_rate
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h, UNNEST(h.hits) hit
WHERE _TABLE_SUFFIX BETWEEN '20170101' AND '20170731'
GROUP BY hour
ORDER BY hour
"""

df = client.query(query).to_dataframe()
df.head()

,hour,total_visits,total_bounces,bounce_rate
0,0,56146,8930,15.904962
1,1,52034,8916,17.134950
2,2,47505,9020,18.987475
3,3,46638,9000,19.297569
4,4,50468,9777,19.372672


In [210]:
# Melt the DataFrame to long format for Plotly express
df_melted = df.melt(id_vars=['hour'], value_vars=['total_visits', 'total_bounces'],
                    var_name='metric', value_name='value')

# Create the line plot
fig = px.line(df_melted, x='hour', y='value', color='metric', markers=True,
              title='Total Visits and Bounces per Hour',
              labels={'value': 'Total visits & Bounces', 'metric': 'Metric'})

# Show the plot
fig.show()

In [211]:
# Create the line plot
fig = px.bar(df, x='hour', y='bounce_rate',
              title='Bounce Rate by Hour',
              labels={'bounce_rate': 'Bounce Rate',})

# Show the plot
fig.show()

NOTE:

Doing this can help us understand better the user behavior and might provide some insights about the best time for creating social media posts with content that might attract more users, such as promotions and discounts.

**Traffic Trends & Patterns**

- By analyzing historical website traffic and sales data over a prolonged period, we can determine whether there are any identifiable patterns that repeat annually or at specific intervals. 

- To analyze seasonal trends, we can aggregate data on a monthly, quarterly, or weekly basis and use visualizations such as line charts, bar graphs, or heatmaps.

- We can also use seasonal decomposition methods, such as moving averages or seasonal indices to extract underlying patterns from the noise.

NOTES:
- "Traffic Source from which the session originated" means the origin of visitors to your website. 
> This information is crucial in understanding how users are finding and accessing your site, which can help optimize marketing efforts and improve user acquisition strategies.


Traffic source can come from following sources:

- Direct: Visitors who arrive at your site by typing the URL directly into their browser, using a bookmark, or other direct means.

- Referral: Visitors who come to your site from a link on another website.

- Organic Search: Visitors who arrive at your site from unpaid search engine results (e.g., Google, Bing).

- Paid Search: Visitors who come to your site through paid advertising on search engines (e.g., Google Ads).

- Social: Visitors who come from social media platforms (e.g., Facebook, Twitter, LinkedIn).

- Email: Visitors who arrive via links in email campaigns.

- Other Campaigns: Visitors from other specific marketing campaigns, such as display ads or affiliate marketing.

In [212]:
query = """
SELECT
  DATE_TRUNC(PARSE_DATE('%Y%m%d', _TABLE_SUFFIX), MONTH) month,
  trafficSource.source source,
  COUNT(*) total_visits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170101' AND '20170731'
GROUP BY
  month, source
ORDER BY
  month, total_visits DESC
"""

df = client.query(query).to_dataframe()
df.head()

,month,source,total_visits
0,2017-01-01,(direct),46795
1,2017-01-01,youtube.com,9709
2,2017-01-01,google,2672
3,2017-01-01,Partners,1248
4,2017-01-01,analytics.google.com,920


In [213]:
fig = px.line(df, x='month', y='total_visits', color='source',
              title='Monthly Traffic Volume by Source',
              markers=True, line_shape='spline')

fig.update_layout(xaxis_title='Month', yaxis_title='Total Visits', legend_title='Source')
fig.show()

As we can see in our line chart dpicting diverse traffic channels, a few salient observations come into focus:

1. **Growth in Organic Search:** The standout trend centers around the sustained growth in organic search traffic. This upward trajectory suggest an encouraging narrative of users organically discovering and engaging with the content.

2. **Fluctuating Direct Traffic:** The direct traffic inflows chart shows fluctuations. Some periods register spikes, others see a dip. The unpredictability of this pattern merits a closer analysis. It would be useful to understand what contributes to the instability in this cannel.

3. **Dimishing YouTube0Sourced Traffic:** Another insight emerges from the diminishing traffic originated from YouTube sources. This decline is a warning to check this channel in detail. Are there shifts in content? Are we changing engagement strategies? Or maybe there are changes in the YouTube algorithm that could be influencing this reduction? Addressing this issue would be good to foster growth again.

Reflecting on these findings, while organic search displays encouraging growth, it's worth considering the prospect of amplifying our focus on YouTube traffic.

**Areas for Improvement**

We've looked closely at our analysis and found pages and products that are doing really well. This information can be really helpful if we can use it to improve the website's search engine optimization (SEO) and making the most of these popular products for marketing purposes.

Importantly, there's a growing interest from people around the world, which means we have a chance to increase sales to international customers, but the complexity of this task depends on a few unknown variables at the moment.

When it comes to our marketing plan, we've figured out what's growing and stable, which helps us decide where to invest our resources, namely YouTube traffic seems to be dwindling and could do better. We should continue the work with channels that are doing well and paying extra attention to the ones that need some improvement.

In a nutshell, these insights sum up our research. With this knowledge, it's possible to adapt the company's approach to better fit customer expectations and expand strategically, engage more with their audience, and maintain a strong presence in the market.

# Source

https://www.kaggle.com/code/leandrocassius/evaluating-store-performance-with-bigquery#Online-Store-Performance:-Insights-from-Google-Analytics-Data-using-BigQuery